# Gradient Calculation

## 1. Introduction

The gradient function uses the Discrete Adjoint with secord-order in time fully-explicit timestepping scheme with implementation of a Perfectly Matched Layer (PML) using CG FEM with or without higher order mass lumping (KMV type elements).

The gradient calculation algorithm follows the steps:
1. a synthetic FWI problem is runned in order to generate a synthetic *shot record* for reference
2. another FWI problem is executed, but with the initial guess
3. the difference between the results is evaluated as follows
$$
misfit = p_{exact_{recv}} - p_{guess_{recv}}
$$
4. then, the misfit and other parameters, such as the mesh, the velocity model guess and the receivers location, are used as input to the `gradient` function

The gradient calculation method is based on the **adjoint method**, because it reduces the computational cost in a problem with many variables. Besides that, the adjoint equations have as source the difference in the receivers between the real values and the guess.

## 2. Example

### 2.1. Real case

The first step is to evaluate the real case. It includes the libraries,

In [1]:
import firedrake as fire
import spyro
import numpy as np

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


model definition,

In [6]:
model = {}

model["opts"] = {
        "method": "KMV",  # either CG or KMV
        "quadrature": "KMV", # Equi or KMV
        "degree": 4,  # p order
        "dimension": 2,  # dimension
        "regularization": False,  # regularization is on?
        "gamma": 1e-5, # regularization parameter
    }

model["parallelism"] = {
        "type": "automatic",  # options: automatic (same number of cores for evey processor) or spatial
    }


model["mesh"] = {
            "Lz": 1.0,  # depth in km - always positive
            "Lx": 1.0,  # width in km - always positive
            "Ly": 0.0,  # thickness in km - always positive
            "meshfile":  None,
            "initmodel": None,
            "truemodel": None,
        }


# Specify a 250-m Absorbing Boundary Layer (ABL) on the three sides of the domain to damp outgoing waves.
model["BCs"] = {
    "status": True,  # True or False, used to turn on any type of BC
    "method": "PML", # either PML or Damping, used to turn on any type of BC
    "outer_bc": "non-reflective", #  none or non-reflective (outer boundary condition)
    "damping_type": "polynomial",  # polynomial, hyperbolic, shifted_hyperbolic
    "exponent": 2,  # damping layer has a exponent variation
    "cmax": 1.5,  # maximum acoustic wave velocity in PML - km/s
    "R": 1e-6,  # theoretical reflection coefficient
    "lz": 0.7,  # thickness of the PML in the z-direction (km) - always positive
    "lx": 0.7,  # thickness of the PML in the x-direction (km) - always positive
    "ly": 0.0,  # thickness of the PML in the y-direction (km) - always positive
}


model["acquisition"] = {
            "source_type": "Ricker",
            "source_pos": [(0.5, 0.5)],
            "frequency": 10.0,
            "delay": 1.0,
            "receiver_locations": spyro.create_transect(
            (0.25, 0.2), (0.25, 0.8), 10
            ),
        }


model["aut_dif"] = {
        "status": True, 
    }

model["timeaxis"] = {
    "t0": 0.0,  #  Initial time for event
    "tf": 1.0,  # Final time for event (for test 7)
    "dt": 0.001,  # timestep size (divided by 2 in the test 4. dt for test 3 is 0.00050)
    "amplitude": 1,  # the Ricker has an amplitude of 1.
    "nspool":  2000,  # (20 for dt=0.00050) how frequently to output solution to pvds
    "fspool": 1,  # how frequently to save solution to RAM
}

mesh generation,

In [7]:
mesh = fire.RectangleMesh(100, 100, 1.5, 1.5) # to test FWI, mesh aligned with interface

model extraction,

In [8]:
def _make_vp_pml(V, mesh, v0=1.5, v1=4.0):
    """Create a half space"""
    z, x = fire.SpatialCoordinate(mesh)
    velocity = fire.conditional(z < 0.5, v0, v1)
    vp       = fire.Function(V, name="vp").interpolate(velocity)
    fire.File("exact_vel.pvd").write(vp)

    return vp

and extraction,

In [9]:
element  = spyro.domains.space.FE_method(
            mesh, model["opts"]["method"], 
            model["opts"]["degree"]
            )
V        = fire.FunctionSpace(mesh, element)
vp_exact = _make_vp_pml(V, mesh)
vp_guess = _make_vp_pml(V, mesh, v0=1.5, v1=1.5)

and model evaluation

In [10]:
comm = spyro.utils.mpi_init(model)
mesh, V = spyro.io.read_mesh(model, comm)
vp = spyro.io.interpolate(model, mesh, V, guess=False)
if comm.ensemble_comm.rank == 0:
    fire.File("true_velocity.pvd", comm=comm.comm).write(vp)
sources = spyro.Sources(model, mesh, V, comm)
receivers = spyro.Receivers(model, mesh, V, comm)
wavelet = spyro.full_ricker_wavelet(
    model["timeaxis"]["dt"],
    model["timeaxis"]["tf"],
    model["acquisition"]["frequency"],
)
p, p_r = spyro.solvers.forward(model, mesh, comm, vp, sources, wavelet, receivers)
spyro.io.save_shots(model, comm, p_r)

1


Error: error code 65
[0] PetscViewerFileSetName() at /home/bruno/libs/firedrake/src/petsc/src/sys/classes/viewer/impls/ascii/filev.c:649
[0] PetscViewerFileSetName_ASCII() at /home/bruno/libs/firedrake/src/petsc/src/sys/classes/viewer/impls/ascii/filev.c:753
[0] Unable to open file
[0] Cannot open PetscViewer file: not_used.msh

### 2.2. Guess model

Similarly, the guess model is submitted to the same process, except for the `save_shots` function.

In [6]:
model = {}

model["opts"] = {
    "method": "KMV",  # either CG or KMV
    "quadrature": "KMV",  # Equi or KMV
    "degree": 4,  # p order
    "dimension": 2,  # dimension
}
model["parallelism"] = {
    "type": "automatic",
}
model["mesh"] = {
    "Lz": ,  # depth in km - always positive
    "Lx": ,  # width in km - always positive
    "Ly": 0.0,  # thickness in km - always positive
    "meshfile": None,
    "initmodel": None",
    "truemodel": None,
}
model["BCs"] = {
    "status": True,  # True or false
    "outer_bc": "non-reflective",  #  None or non-reflective (outer boundary condition)
    "damping_type": "polynomial",  # polynomial, hyperbolic, shifted_hyperbolic
    "exponent": 2,  # damping layer has a exponent variation
    "cmax": 4.5,  # maximum acoustic wave velocity in PML - km/s
    "R": 1e-6,  # theoretical reflection coefficient
    "lz": ,  # thickness of the PML in the z-direction (km) - always positive
    "lx": ,  # thickness of the PML in the x-direction (km) - always positive
    "ly": 0.0,  # thickness of the PML in the y-direction (km) - always positive
}
model["acquisition"] = {
    "source_type": "Ricker",
    "source_pos": ,
    "frequency": 5.0,
    "delay": 1.0,
    "receiver_locations": ,
}
model["timeaxis"] = {
    "t0": 0.0,  #  Initial time for event
    "tf": 5.00,  # Final time for event
    "dt": 0.001,
    "amplitude": 1,  # the Ricker has an amplitude of 1.
    "nspool": 100,  # how frequently to output solution to pvds
    "fspool": 99999,  # how frequently to save solution to RAM
}

In [7]:
comm = spyro.utils.mpi_init(model)
element  = spyro.domains.space.FE_method(
            mesh, model["opts"]["method"], 
            model["opts"]["degree"]
            )
V        = FunctionSpace(mesh, element)
vp = spyro.io.interpolate(model, mesh, V, guess=True)
if comm.ensemble_comm.rank == 0:
    File("true_velocity.pvd", comm=comm.comm).write(vp)
sources = spyro.Sources(model, mesh, V, comm)
receivers = spyro.Receivers(model, mesh, V, comm)
wavelet = spyro.full_ricker_wavelet(
    dt=model["timeaxis"]["dt"],
    final_time=model["timeaxis"]["tf"],
    frequency=model["acquisition"]["frequency"],
)
p, p_r = spyro.solvers.forward(model, mesh, comm, vp, sources, wavelet, receivers)

1


ValueError: Available cores cannot be divided between sources equally.

If there is water inside the domain, it should be localized:

In [10]:
water = np.where(vp.dat.data[:] < 0.5)

NameError: name 'vp' is not defined

### 2.3. Gradient calculation

The gradient is calculated according the adjoint method. The receivers are acting as sources and the misfit is the recorded value of the sources:

In [12]:
dJ = Function(V, name="gradient")
dJ_local = spyro.solvers.gradient(
    model,
    mesh,
    comm,
    vp,
    receivers,
    self.p_guess,
    self.misfit,
)
if comm.ensemble_comm.size > 1:
    comm.allreduce(dJ_local, dJ)
else:
    dJ = dJ_local
dJ /= comm.ensemble_comm.size
if comm.comm.size > 1:
    dJ /= comm.comm.size
# regularize the gradient if asked.
if model["opts"]["regularization"]:
    dJ = regularize_gradient(vp, dJ)
# mask the water layer
dJ.dat.data[water] = 0.0

NameError: name 'V' is not defined